Moving forward, we will be using google colab to run our code. You can access google colab with this link: https://colab.research.google.com/

# Dataframes and data visualization

## Setup Steps

This assumes you have a working Jupyter setup on your local machine. If you don't have these packages installed, go ahead and open the "Anaconda Navigator" application and install these packages: plotly, pandas, numpy, matplotlib. You may need to reload your VS Code window after installation for those packages to work.

This is a huge help and a number of our cells come from it! https://www.kaggle.com/code/orhansertkaya/visualizing-pok-mon-stats-with-plotly/notebook



## Loading data into Pandas

If you are using repl, copy the contents of the csv into your repl and give it the same name of `pokemon_data.csv`

In [ ]:
import pandas as pd

# Read csv data
df = pd.read_csv('pokemon_data.csv')

Pandas can easily read from different file formats. Here are some examples of what it looks like to read from other files. Don't run this code as the files are not defined - it will throw you an error.

In [ ]:
# Example of reading excel data
# df_xlsx = pd.read_excel('pokemon_data.xlsx')
# df_xlsx.head()

# Example of reading txt data
# df_txt = pd.read_csv('pokemon_data.txt', delimiter='\t')
# df_txt.head()

## Understanding Data in Pandas

Here you will some sample rows from your dataset using the .head() function.

In [ ]:
# Read the first 5 rows by default, if you want a different 
# number of rows pass it as an argument to the head function.
df.head()

Understanding your data with pandas can start out in a couple ways. First we can find out the different columns in the dataframe using the .columns() function.

In [ ]:
# Read headers
df.columns

We can also select certain columns and display all the row values associated with them by calling the dataframe and in square brackets inserting the names of columns you want the row data called for.

In [ ]:
# Read these columns from all rows
print(df[['Name', 'Type 1', 'HP']])

<span style ="background-color:yellow">
TODO: Find 2 columns and show all the values in those columns.
</span>

To return the first 4 rows we would use the .iloc[] function in order to retrieve that data with a specific slice inserted within the square brackets. 'df.iloc[0:4]', retrieves the first 4 rows.

In [ ]:
print(df.iloc[0:4])

The for loop allows us to access every row and for every row in our data set we are printing the string "new row". Then we are printing the index followed by the data that is associated with the row and this would all be ordered by the pokemon name.

In [ ]:
# Read every row
for index, row in df.iterrows():
    print("\nNew Row")
    print(index, row)

The .iloc[] function is important for returning specific values using their location. By changing the inputs and the amount of inputs you will be able to return entire rows of data or singular cells. Below are 4 examples of using the .iloc[] function returning different outputs.

In [ ]:
# Read a specific row's column (row 10 column 2 value)
print(df.iloc[10, 2])

# Read a range of rows
print(df.iloc[1:5])

# Read specific column from a range of rows
print(df.iloc[1:5, 1])

# Read range of columns from a range of rows
print(df.iloc[1:5, 1:3])

<span style ="background-color:yellow">
TODO: Using the .iloc[] choose 5 rows you want displayed and describe to a classmate in a couple sentences how one could use the output to help understand the data better.
</span>

## Sampling Data


This section starts with the .value_counts() function where it aggregates data to make it easier to read. In the below cell, we want to find out the amount of pokemon in the dataset that belongs to each generation. We start out by specifying what dataframe we want to use and then input the column we are aggregating by.

In [ ]:
# Get a count for each unique value in the 'Generation' column
df.Generation.value_counts()

<span style ="background-color:yellow">
TODO: Describe how the .value_counts() function could be used to aggregate data in your dataset in a useful way. Then aggregate the data using that function and see if it worked in the way it was intended to. 
</span>

In case we want to find out how many pokemon are considered legendary we can use .sum() after specifying the column we want to aggregate by.

In [ ]:
# Calculate the total number of pokemon that are considered legendary.
Legendary = df['Legendary'].sum()
print(f"Pokemon that are considered legendary: {Legendary}")

To find the total pokemon we can use the number of unique 'IDs' there are in the ID column which in this case is declared using a pound sign. Then we use the .count function since we want to find the total count of IDs.

In [ ]:
# Calculate total number of respondents using the count of 'ID' column (assuming each ID is unique).
total_pokemon = df['#'].count()
print(f"Total pokemon: {total_pokemon}")

## Sorting/Describing Data

Sorting and describing data is important to figure out what the data looks like with certain filters applied. Below is an example of sorting pokemon by their type alphabetically and the amount of HP they have. First pokemon are ordered alphabetically by 'Type 1' (ex: Ghost before Water). Then pokemon are ordered numerically by 'HP' (ex: 35 before 60).

In [ ]:
# Sort by type and health points (HP) so type is alphabetical ascending and 
# hp is numeric ascending. Ascending is either true or false.

df.sort_values(by=['Type 1', 'HP'],ascending=True)


<span style ="background-color:yellow">
TODO: List the 3 best columns that could be used to sort the values in a useful way. Provide a one sentence explanantion on why these columns would help someone interpret the data better. Lastly, use the .sort_values() function to sort the data in the desired way.
</span>

The .groupby() function is a powerful tool that allows data to be sorted and aggregated in whatever way you choose. In the case below the pokemon types are used in order to group the pokemon. 

In [ ]:
# Group the data by 'Type 1' and count the values of 'Type 2' in each group.
df.groupby('Type 1')['Type 2'].value_counts()

## Making changes to the data

Inserting new columns and dropping columns based on present data is important for reading code as it can provide more information to the user and allows more ways for data sorting. In this case a column called 'Total' is made using various other columns by adding up the values of other columns. All the columns must by in a numberic data type in order for this to work. Another column, 'Sp. Power', is made and then dropped by using the .drop() function and then specifying which column should be dropped.

In [ ]:
# Create a new column in each row based on a calculation that would be performed on every row
df['Total'] = df['HP'] + df['Attack'] + df['Defense'] + df['Sp. Atk'] + df['Sp. Def'] + df['Speed']

# Checkout the new column
df.head()

# Create a new column again
df['Sp. Power'] = df['Sp. Atk'] + df['Sp. Def']

# Drop the newly created column
df = df.drop(columns=['Sp. Power'])

# The column should be gone now
df.head()

<span style ="background-color:yellow">
TODO: Make 2 columns that would be beneficial to interpreting the data. Provide a sentence explanantion on what is useful about the information provided by these new columns. Afterwards drop 1 of the columns and provide an explanation on why the remaining added column was better to leave in. 
</span>

## Saving our Data (Exporting into Desired Format)

As we described the earlier in the lesson, pandas can easily read from other file formats and read data. However, it can also __export__ to these different formats just as easily.

In [ ]:
# df.to_csv('modified.csv', index=False)

# df.to_excel('modified.xlsx', index=False)

# df.to_csv('modified.txt', index=False, sep='\t')

## Filtering Data

Filtering data involves finding what is nessessary, manipulating it to showcase it in an easy to read way and often saving it. Below the .loc() function is used to find what data is wanted. Then that filtered data is assigned to a new dataframe. By using the.reset_index() function we can use the original index and use it as a column in our new 'filtered.csv' file.

In [ ]:
# Note that in dataframes there is no "and", only &. Lets filter for some data.
new_df = df.loc[(df['Type 1'] == 'Grass') & (df['Type 2'] == 'Poison') & (df['HP'] > 70)]

# Reassign the row numbers
new_df.reset_index(drop=True, inplace=True)

new_df.head()

# Save this filtered data to a new csv
new_df.to_csv('filtered.csv')



## Conditional Changes

Sometimes we want to change values based on conditions. Below if a Legendary pokemon is found they will be labeled with 'Test 2' instead of 'TRUE'. Then we display the column.

In [ ]:

# Conditionally change a column
df.loc[df['Total'] > 500, ['Generation','Legendary']] = ['Test 1', 'Test 2']

# Display the column
df['Legendary']

## Aggregate Statistics (Groupby)


Aggregating statistics using .groupby() involves creating a new column known as count with an assigned value of 1. Then we populate this column using the 
.groupby() function and inputing how the data should be grouped. The assigned value in the 'count' column is changed to whatever count is produced using the .groupby function.

In [ ]:
# Create new column with every value initialized to 1
df['count'] = 1

# Group and display a column's count based on number of Type 1 and Type 2 pokemon
df.groupby(['Type 1', 'Type 2']).count()['count']

<span style ="background-color:yellow">
TODO: Using the .groupby() and the .count() functions determine what columns could be best aggregated in a similar method as the cell above. Remember you must create a new column to aggregate that data in.
</span>

## Working with large amounts of data



Sometimes sorting through all the data is too much, so chunking the data is very helpful in order to make easier to read. There are many ways to chunk data but in this case the data is chunked by type and represented as a count. There are other ways to chunk such as using location, columns names, etc...

In [ ]:
new_df = pd.DataFrame(columns=df.columns)

# Chunk a dataframe from a csv
for dfr in pd.read_csv('pokemon_data.csv', chunksize=5):
    results = dfr.groupby(['Type 1']).count()
    new_df = pd.concat([new_df, results])
new_df.head()